In [1]:
import tweepy       # python wrapper for twitter api
import json
import time
import requests
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer

# step 0, get your own twitter credentials!

In [2]:
# this is my own personal twitter api information
# if you could be so kind as to sign up yourself on both twitter and mashape that'd be great :)
# It's FREEEEEEE
api_key = 'g3xAZVinye67J9guxFhGT5Y8f'
api_secret = 'TYkqHGyBxNSoziaap8LtygafZpD6cHjLuiIdgfbvNRr3RV2fwU'
access_token = '536799553-4jpBdlg8XMtnXCVEA2daxrDYuwT4HTQNx7eEOtoJ'
access_secret = 'Mya4FnubounjlXM1Vh4fwDTLwOjIT6UbTNiBzQQUEAJqI'

In [3]:
# Documentation is your friend! http://docs.tweepy.org/en/v3.1.0/
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth) # returns a tweepy authorization handler object

In [4]:
# 1. Use Twitter API to stream and store tweets (300 per candidate)
# explain geocode is doing
bernie_tweets =  api.search(q="#berniesanders", count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#bernie', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#FeelTheBern', count=100, geocode="37.781157,-122.398720,50mi")
hilary_tweets = api.search(q='#ImWithHer', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#hillaryclinton', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#hillary2016', count=100, geocode="37.781157,-122.398720,50mi")
the_donald_tweets = api.search(q='#donaldtrump', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#trump2016', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#trump', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#makeamericagreatagain', count=100, geocode="37.781157,-122.398720,50mi")

Geocode is returning only tweets from within 50 miles of the given coordinates

In [5]:
the_donald_tweets[0]

Status(contributors=None, truncated=False, text=u'RT @JacloPac: Steps must be taken to prevent #DonaldTrump &amp; the #GOP from the White House....... https://t.co/Hfd50hyjJ6', is_quote_status=False, in_reply_to_status_id=None, id=730247362489233409, favorite_count=0, _api=<tweepy.api.API object at 0x10ec96bd0>, author=User(follow_request_sent=False, has_extended_profile=True, profile_use_background_image=True, _json={u'follow_request_sent': False, u'has_extended_profile': True, u'profile_use_background_image': True, u'default_profile_image': False, u'id': 2287118126, u'profile_background_image_url_https': u'https://abs.twimg.com/images/themes/theme1/bg.png', u'verified': False, u'profile_text_color': u'333333', u'profile_image_url_https': u'https://pbs.twimg.com/profile_images/422105640056537088/5Nh_Fnjx_normal.jpeg', u'profile_sidebar_fill_color': u'DDEEF6', u'entities': {u'description': {u'urls': []}}, u'followers_count': 149, u'profile_sidebar_border_color': u'C0DEED', u'id_str': u

In [6]:
# wrappers come with built in python attributes and methods!
print dir(the_donald_tweets[0])

['__class__', '__delattr__', '__dict__', '__doc__', '__eq__', '__format__', '__getattribute__', '__getstate__', '__hash__', '__init__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_api', '_json', 'author', 'contributors', 'coordinates', 'created_at', 'destroy', 'entities', 'favorite', 'favorite_count', 'favorited', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'metadata', 'parse', 'parse_list', 'place', 'possibly_sensitive', 'retweet', 'retweet_count', 'retweeted', 'retweeted_status', 'retweets', 'source', 'source_url', 'text', 'truncated', 'user']


In [7]:
print the_donald_tweets[0].created_at
print the_donald_tweets[0].text
print the_donald_tweets[0].favorite_count
print the_donald_tweets[0].favorited
print the_donald_tweets[0].lang
print the_donald_tweets[0].geo
print the_donald_tweets[0].retweet_count
print the_donald_tweets[0].retweeted
print the_donald_tweets[0].coordinates

2016-05-11 04:05:13
RT @JacloPac: Steps must be taken to prevent #DonaldTrump &amp; the #GOP from the White House....... https://t.co/Hfd50hyjJ6
0
False
en
None
25
False
None


In [10]:
def parse_status_objects(tweet_list,candidate, party):
    modified_tweets = list()
    for x in tweet_list:
        modified_tweets.append( [
                    x.created_at,
                    x.text,
                    x.favorite_count,
                    x.favorited,
                    x.lang,
                    x.geo,
                    x.retweet_count,
                    x.retweeted,
                    x.coordinates,
                    candidate,
                    party
                ])
    return modified_tweets
        
hilary_tweets = parse_status_objects(hilary_tweets,0,1)
the_donald_tweets = parse_status_objects(the_donald_tweets,2,0)
bernie_tweets = parse_status_objects(bernie_tweets,1,1)

In [11]:
print bernie_tweets[0]
print the_donald_tweets[0]
print hilary_tweets[0]

[datetime.datetime(2016, 5, 11, 4, 18, 51), u'Endorsements @sfbg #BernieSanders @JaneKim for #SD11 #YesOnC #ReformDemocrats #CAPrimary #sfpolitics #RunjaneRun  https://t.co/8nvhzrnW7K', 0, False, u'und', None, 0, False, None, 1, 1]
[datetime.datetime(2016, 5, 11, 4, 5, 13), u'RT @JacloPac: Steps must be taken to prevent #DonaldTrump &amp; the #GOP from the White House....... https://t.co/Hfd50hyjJ6', 0, False, u'en', None, 25, False, None, 2, 0]
[datetime.datetime(2016, 5, 11, 4, 13, 9), u'Another reason @HllaryClinton is the stronger #Democratic candidate: Pro-HRC super-PACs are necessary to define &amp; stop Trump. #ImWithHer', 0, False, u'en', None, 0, False, None, 0, 1]


In [18]:
# 2 Create a dataframe that combines all of the tweets from each candidate
# To do so, you will need to concat the data frames
# so each row is a tweet and your columns should be 
# date, text, favorite_count,favorited, language, geocode, rwtweet count, retweeted, coordinates

Bernie = pd.DataFrame(data = bernie_tweets, columns = ['date', 'text', 'favorite_count', 'favorited', 'language', 
                                                       'geocode', 'rwtweet count', 'retweeted', 'coordinates','candidate','party'])

Donald = pd.DataFrame(data = the_donald_tweets, columns = ['date', 'text', 'favorite_count', 'favorited', 'language', 
                                                       'geocode', 'rwtweet count', 'retweeted', 'coordinates','candidate','party'])

Hilary = pd.DataFrame(data = hilary_tweets, columns = ['date', 'text', 'favorite_count', 'favorited', 'language', 
                                                       'geocode', 'rwtweet count', 'retweeted', 'coordinates','candidate','party'])



df = pd.concat([Bernie, Hilary, Donald], axis = 0)
pd.set_option('max_colwidth', 500)


df = df.reset_index()

In [13]:
# 3 Create a function to take in a string and output the textblob sentiment of that string
def sentiment(string):
    tweet = TextBlob(string)
    return tweet.sentiment.polarity

In [14]:
# 4 Add a column to your data frame called 'sentiment' which holds the sentiment of that tweet (hint: use the function from #3)
df['sentiment'] = df.text.apply(sentiment)

In [ ]:
#df[0]?

In [15]:
# 5 create a word count column, which holds the number of words in the text column
def wordcount(string):
    tweet = TextBlob(string)
    return len(tweet.words)

df['word_count'] = df.text.apply(wordcount)
df.head(20)

,index,date,text,favorite_count,favorited,language,geocode,rwtweet count,retweeted,coordinates,candidate,party,sentiment,word_count
0,0,2016-05-11 04:18:51,Endorsements @sfbg #BernieSanders @JaneKim for #SD11 #YesOnC #ReformDemocrats #CAPrimary #sfpolitics #RunjaneRun https://t.co/8nvhzrnW7K,0,False,und,None,0,False,None,1,1,0.000000,13
1,1,2016-05-11 04:18:44,"RT @LiLightfoot: It takes honesty, integrity, compassion, and common sense to inspire the people! #BernieSanders Congrats on your win https…",0,False,en,None,10,False,None,1,1,0.212500,20
2,2,2016-05-11 04:10:24,Register or re-register to change party affiliation. #CAPrimary #LosAngeles #BernieSanders https://t.co/hGiLDonjaU,0,False,en,None,0,False,None,1,1,0.000000,12
3,3,2016-05-11 04:00:31,"RT @LiLightfoot: It takes honesty, integrity, compassion, and common sense to inspire the people! #BernieSanders Congrats on your win https…",0,False,en,None,10,False,None,1,1,0.212500,20
4,4,2016-05-11 03:56:50,"RT @LiLightfoot: It takes honesty, integrity, compassion, and common sense to inspire the people! #BernieSanders Congrats on your win https…",0,False,en,None,10,False,None,1,1,0.212500,20
5,5,2016-05-11 03:55:57,"RT @LiLightfoot: It takes honesty, integrity, compassion, and common sense to inspire the people! #BernieSanders Congrats on your win https…",0,False,en,None,10,False,None,1,1,0.212500,20
6,6,2016-05-11 03:55:35,"RT @LiLightfoot: It takes honesty, integrity, compassion, and common sense to inspire the people! #BernieSanders Congrats on your win https…",0,False,en,None,10,False,None,1,1,0.212500,20
7,7,2016-05-11 03:45:23,RT @LiLightfoot: #BernieSanders is just ringing his hands wondering how he's going to survive the devastating Victory in WestVirginia https…,0,False,en,None,5,False,None,1,1,-1.000000,21
8,8,2016-05-11 03:41:20,The Bernie Rally in Sacramento. #GoBernie! #BernieSanders #Election2016 https://t.co/aR8K7ExA9n,0,False,en,None,0,False,None,1,1,0.000000,10
9,9,2016-05-11 03:39:59,"RT @LiLightfoot: It takes honesty, integrity, compassion, and common sense to inspire the people! #BernieSanders Congrats on your win https…",0,False,en,None,10,False,None,1,1,0.212500,20


In [16]:
# use countvectorizer to create a document-term matrix and concatinate it to your main dataframe
#(hint you should now have thousands of columns)

vect = CountVectorizer(ngram_range = (1,2), stop_words='english', min_df =2)
doc_matrix = pd.DataFrame(vect.fit_transform(df.text).toarray(), columns=vect.get_feature_names())
type(df)


pandas.core.frame.DataFrame

In [17]:
print doc_matrix.index
print df.index
len(df.index)

RangeIndex(start=0, stop=914, step=1)
RangeIndex(start=0, stop=914, step=1)


914

In [20]:
pres_tweets = pd.concat([df, doc_matrix], axis=1)
pres_tweets.head()

,index,date,text,favorite_count,favorited,language,geocode,rwtweet count,retweeted,coordinates,...,yolibeans,yolibeans know,young,youtube,yrs,yuge,yuge margin,yz3yn,zuckerberg,zuckerberg censoring
0,0,2016-05-11 04:18:51,Endorsements @sfbg #BernieSanders @JaneKim for #SD11 #YesOnC #ReformDemocrats #CAPrimary #sfpolitics #RunjaneRun https://t.co/8nvhzrnW7K,0,False,und,None,0,False,None,...,0,0,0,0,0,0,0,0,0,0
1,1,2016-05-11 04:18:44,"RT @LiLightfoot: It takes honesty, integrity, compassion, and common sense to inspire the people! #BernieSanders Congrats on your win https…",0,False,en,None,10,False,None,...,0,0,0,0,0,0,0,0,0,0
2,2,2016-05-11 04:10:24,Register or re-register to change party affiliation. #CAPrimary #LosAngeles #BernieSanders https://t.co/hGiLDonjaU,0,False,en,None,0,False,None,...,0,0,0,0,0,0,0,0,0,0
3,3,2016-05-11 04:00:31,"RT @LiLightfoot: It takes honesty, integrity, compassion, and common sense to inspire the people! #BernieSanders Congrats on your win https…",0,False,en,None,10,False,None,...,0,0,0,0,0,0,0,0,0,0
4,4,2016-05-11 03:56:50,"RT @LiLightfoot: It takes honesty, integrity, compassion, and common sense to inspire the people! #BernieSanders Congrats on your win https…",0,False,en,None,10,False,None,...,0,0,0,0,0,0,0,0,0,0


In [28]:
X_clusters.dtypes

index                                int64
date                        datetime64[ns]
favorite_count                       int64
favorited                             bool
language                            object
geocode                             object
rwtweet count                        int64
retweeted                             bool
coordinates                         object
000                                  int64
000 doctors                          int64
0p0orubcll                           int64
0p0orubcll stoptpp                   int64
10                                   int64
10 mins                              int64
1001                                 int64
1001 van                             int64
107                                  int64
107 delegates                        int64
10am                                 int64
10am 9pm                             int64
13                                   int64
13 minutes                           int64
1457       

In [30]:
# 6 Use kmean to create clusters for the dataframe
X_clusters = pres_tweets.drop(['text','candidate', 'party', 'language', 'geocode', 'coordinates'], axis =1)
                               
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3, random_state=1)
km.fit(X_clusters)

TypeError: float() argument must be a string or a number

In [ ]:
pres_tweets['cluster'] = km.labels_
pres_tweets.sort('cluster')

In [ ]:
#calculate the Silhouette Coefficient
from sklearn import metrics
metrics.silhouette_score(X_clusters, km.labels_)

In [ ]:
#tune the parameters
k_range = range(2, 20)
scores = []
for k in k_range:
    km = KMeans(n_clusters=k, random_state=1)
    km.fit(X_clusters)
    scores.append(metrics.silhouette_score(X_scaled, km.labels_))
    
# plot the results
plt.plot(k_range, scores)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')
plt.grid(True)

In [ ]:
# K-means with # clusters on scaled data
km = KMeans(n_clusters=#, random_state=1)
km.fit(X_clusters)

In [ ]:
# 7 Explore the clusters
# What is interesting? Surprising? Can you draw any inferences about each cluster?
# Write your answer to these questions in markdown

In [ ]:
pres_tweets['cluster'] = km.labels_
pres_tweets.sort('cluster')

In [ ]:
centers = pres_tweets.groupby('cluster').mean()

In [ ]:
import numpy as np
colors = np.array(['red', 'green', 'blue', 'yellow'])

pd.scatter_matrix(X_clusters, c= colors[list(pres_tweets.cluster)], figsize=(10,10), s=100)

In [ ]:
# 8 Create a column called 'democrat' and assign it to the corresponding candidate name for all the records
# 1 for bernie and hilary and 0 for donald trump

In [ ]:
# 9 Estimate classification models using knn and logistic regession.  Dont forget to use cross validation 
    # (do not use cross_val_score because we will want a confusion matrix).
# Use a three class response variable for knn.  For logistic regression predict democrat or republic?
#Assess the pros and cons of both approaches
#Extra Credit: Estimate multinomial regression model using the three class response variable

In [ ]:
# split into training and testing sets
from sklearn.cross_validation import train_test_split
lgX_train, lgX_test, lgy_train, lgy_test = train_test_split(X_clusters, pres_tweets.party, random_state=1)

knnX_train, knnX_test, knny_train, knny_test = train_test_split(X_clusters, pres_tweets.candidate, random_state=1)

In [ ]:
# 10 Initialize a classification model (for both knn and logistic)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
# 11 Train your classification model  (for both knn and logistic)

In [ ]:
logreg.fit(lgX_train, lgy_train)
logreg.score(lgX_train, lgy_train)

In [ ]:
knn.fit(knnX_train, knny_train)

scores = []
for i in range(1, 30):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X, y)
    scores.append(knn.score(X, y))
    
plt.plot(range(1,30), scores)
plt.ylabel("Accuracy")
plt.xlabel("K")

scores.max()

In [ ]:
# 12 Predict on your test features and targets (for both knn and logistic)

In [ ]:
lgpreds = logreg.predict(lgX_test)

In [ ]:
knnpreds = knn.predict(knnX_test)

In [ ]:
# 13 output your accuracy (for both knn and logistic)

In [ ]:
from sklearn import metrics
print metrics.accuracy_score(lgy_test, lgpreds)

In [ ]:
# compare to null accuracy rate
lgy_test.mean()
1 - lgy_test.mean()

In [ ]:
print metrics.accuracy_score(knny_test, knnpreds)

In [ ]:
# 14 Create a confusion matrix for your predictions (for both knn and logistic)

In [ ]:
from nltk import ConfusionMatrix
print ConfusionMatrix(list(lgy_test), list(lgpreds))

In [ ]:
print ConfusionMatrix(list(knny_test), list(knnpreds))

In [ ]:
# 15 Who is the easiest candidate to predict? Why do you think? Who is the hardest? Why do you think?
# Write your answers in markdown (for both knn and logistic)

In [ ]:
# 16 What are some extra features (data points) you would like to add to this data set. Explain in markdown.
# (hint maybe something like a boolean, "has_hashtag")

In [ ]:
# 17 Try adding extra features to improve the quality of your predictions (for both knn and logistic)